In [ ]:
import datetime
import os 
import random

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_percentage_error, mean_absolute_error
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.utils import shuffle

from keras.models import Sequential
from keras.layers import LSTM,Dense,Flatten, Conv1D, MaxPooling1D
from keras.optimizers import Adam, SGD


In [ ]:
#stations' data preparation
 
hour_intervals=[1,2,6,12,18,24,]   # end of this cell 0 is added for original input dataset
hour_steps_for_height=39 #39 is the number of different heights

result_folder="results and todos \\"
data_folder="data\\"
stations=["BJNM2009_foF2.txt","ISTA2009_foF2.txt","KGNI2009_foF2.txt","NRIL2009_foF2.txt","WUHN2009_foF2.txt",\
          "BJNM2012_foF2.txt","ISTA2012_foF2.txt","KGNI2012_foF2.txt","NRIL2012_foF2.txt","WUHN2012_foF2.txt",\
          "BJNM2015_foF2.txt","ISTA2015_foF2.txt","KGNI2015_foF2.txt","NRIL2015_foF2.txt","WUHN2015_foF2.txt",\
          "BJNM2019_foF2.txt","ISTA2019_foF2.txt","KGNI2019_foF2.txt","NRIL2019_foF2.txt","WUHN2019_foF2.txt"]

stations_data={}

for st in stations:
    stations_data[st]=pd.read_csv(data_folder+st, sep =',').iloc[::hour_steps_for_height]
    print("\n\n--------------------------"+st+"--------------------------------------\n")
    print(stations_data[st].shape)
    print(stations_data[st].head(1))
    

In [ ]:
#Input-Output data preparation,split and scaling for ML Models

X_train_base,X_val_base,X_test_base, y_train_base,y_val_base, y_test_base={},{},{},{},{},{}

num_fold=4

train_ratio=0.8
val_ratio=0.1
test_ratio=0.1

scaler = MinMaxScaler()

for st in stations_data:
    
    X, y = {}, {}
    X[st] = stations_data[st].drop(['Date','Hour','DOY','Height','F10.7','EqDistance','foF2'], axis=1).to_numpy()
    y[st] = stations_data[st]['foF2'].to_numpy()

    # k-fold train_test split 
    X_train_base[st],X_val_base[st],X_test_base[st], y_train_base[st],y_val_base[st], y_test_base[st] \
        = np.empty((0,X[st].shape[1])),np.empty((0,X[st].shape[1])),np.empty((0,X[st].shape[1])),np.empty((0,)),np.empty((0,)),np.empty((0,))

    for i in range(num_fold):

        len_fold=int(X[st].shape[0]/num_fold)
        split_index_train = len_fold*i+int(train_ratio * len_fold)
        split_index_val = split_index_train+int(val_ratio * len_fold)


        X_train_base[st]=np.vstack([X_train_base[st],X[st][i*len_fold:split_index_train]])
        y_train_base[st]=np.hstack([y_train_base[st],y[st][i*len_fold:split_index_train]])
        
        X_val_base[st]=np.vstack([X_val_base[st],X[st][split_index_train:split_index_val]])
        y_val_base[st]=np.hstack([y_val_base[st],y[st][split_index_train:split_index_val]])
        
        X_test_base[st]=np.vstack([X_test_base[st],X[st][split_index_val:(i+1)*len_fold]])
        y_test_base[st]=np.hstack([y_test_base[st],y[st][split_index_val:(i+1)*len_fold]])


    #shapes
    print(st)
    print("Train: ",X_train_base[st].shape,y_train_base[st].shape,"Validation:",X_val_base[st].shape,y_val_base[st].shape,"Test: ",X_test_base[st].shape,y_test_base[st].shape,"\n")
    
    #scaling
    scaler.fit(X_train_base[st])
    X_train_base[st] = scaler.transform(X_train_base[st])
    X_val_base[st] = scaler.transform(X_val_base[st])
    X_test_base[st] = scaler.transform(X_test_base[st])

    print("Scaled X_train: ",X_train_base[st][0],"\n") #prints first row of X_train



In [ ]:
# Print or save results
result_array = [['Train', 'Train', 'Train', 'Train','Validation', 'Validation', 'Validation', 'Validation', 'Test', 'Test', 'Test', 'Test'],
          ['RMSE', 'R2', 'MAE', 'MAPE','RMSE', 'R2', 'MAE', 'MAPE','RMSE', 'R2', 'MAE', 'MAPE']]
results_df=pd.DataFrame(index=pd.MultiIndex.from_arrays(result_array,names=["ExperimentData","Metric"]))
save_row_index=0
save_column_index=0


def save_error_to_df(model_name, data_type, mse_result,r2_result,mae_result,mape_result):
    
    results_df.loc[(data_type, 'RMSE'), model_name] = mse_result
    results_df.loc[(data_type, 'R2'), model_name] = r2_result
    results_df.loc[(data_type, 'MAE'), model_name] = mae_result
    results_df.loc[(data_type, 'MAPE'), model_name] = mape_result

def printError(model_name, data_type, y_in, pred_in):
    # Calculate ensemble metrics
    print("\n",model_name + " " + data_type +' Model Performance:'+"\n")
    _rmse = np.sqrt(mean_squared_error(y_in, pred_in))
    _r2 = r2_score(y_in, pred_in)
    _mae = mean_absolute_error(y_in, pred_in)
    _mape = np.mean(np.abs((y_in - pred_in) / y_in)) * 100
    
    print('RMSE:', _rmse)
    print('R^2:', _r2)
    print('MAE:', _mae)
    print('MAPE:', _mape)

    save_error_to_df(model_name,data_type,round(_rmse,2),round(_r2,2),round(_mae,2),round(_mape,2))

def save_error_to_excel(st_result_header,first_run):
    global save_row_index
    global save_column_index
    st_header_df=pd.DataFrame(data=st_result_header)

    if first_run:
        #Export all results to an excel file
        st_header_df.to_excel(result_folder+"foF2_ML_Experiments.xlsx",startrow=save_row_index,startcol=save_column_index,index=False,header=False)
        save_row_index+=1
      
        with pd.ExcelWriter(result_folder+"foF2_ML_Experiments.xlsx", mode="a",if_sheet_exists="overlay") as writer:
            results_df.to_excel(writer,startrow=save_row_index,startcol=save_column_index)
        save_row_index+=16
    else :
        with pd.ExcelWriter(result_folder+"foF2_ML_Experiments.xlsx", mode="a",if_sheet_exists="overlay") as writer:
            st_header_df.to_excel(writer,startrow=save_row_index,startcol=save_column_index,index=False,header=False)
            save_row_index+=1
            results_df.to_excel(writer,startrow=save_row_index,startcol=save_column_index)
            save_row_index+=16

def generate_mape_elem(st, model, mape):
    my_dict = {
        "st": st,
        "model": model,
        "mape": mape
    }
    return my_dict

def generate_lstm_result(st, hour,y_real,y_pred):
    my_dict = {
        "st": st,
        "hour": hour,
        "real": y_real,
        "pred": y_pred
    }
    return my_dict


In [ ]:
first_run=True
mape_result_array=[]
lstm_results=[]

for hour_interval in hour_intervals:
    
    for st in stations_data:

        st_result_header=[st[:st.rfind('_')]+"_"+str(hour_interval)+"h"] #Results' header

        # data preparation for different hours

        X_train = X_train_base.copy()[st][:-hour_interval]
        X_test = X_test_base.copy()[st][:-hour_interval]
        X_val = X_val_base.copy()[st][:-hour_interval]

        y_train = y_train_base.copy()[st][hour_interval:]
        y_test = y_test_base.copy()[st][hour_interval:]
        y_val = y_val_base.copy()[st][hour_interval:]
        
        print("\nSTATION: ",st[:-9]," HOUR: ",hour_interval)

        #Linear Regression Model

        reg = LinearRegression().fit(X_train, y_train)

        reg_predTrain = reg.predict(X_train) 
        reg_errorTrain = y_train - reg_predTrain
        printError("Linear Regression","Train", y_train, reg_predTrain)

        reg_predVal = reg.predict(X_val) 
        reg_errorVal = y_val - reg_predVal
        printError("Linear Regression","Validation", y_val, reg_predVal)

        reg_predTest = reg.predict(X_test)
        reg_errorTest = y_test - reg_predTest
        printError("Linear Regression","Test", y_test, reg_predTest)

        mape_result_array.append(generate_mape_elem(st,"Linear Regression",np.mean(np.abs(reg_errorTest / y_test)) * 100))


         #Decision Tree Model 
        dectree = DecisionTreeRegressor(max_depth=12,min_samples_leaf=5,random_state=44).fit(X_train, y_train)

        dectree_predTrain = dectree.predict(X_train) 
        dectree_errorTrain = y_train-dectree_predTrain
        printError("Decision Tree","Train", y_train ,dectree_predTrain)

        dectree_predVal = dectree.predict(X_val) 
        dectree_errorVal = y_val - dectree_predVal
        printError("Decision Tree","Validation", y_val, dectree_predVal)

        dectree_predTest = dectree.predict(X_test)
        dectree_errorTest = y_test - dectree_predTest
        printError("Decision Tree","Test", y_test, dectree_predTest)

        mape_result_array.append(generate_mape_elem(st,"Decision Tree",np.mean(np.abs(dectree_errorTest / y_test)) * 100))

        # MLP model

        #default parameters= loss:mse, learning_rate:adaptive, batch:200
        mlp = MLPRegressor(hidden_layer_sizes=(12,4), activation='relu',batch_size=72, solver='adam', max_iter=1000,shuffle=False,random_state=44)
        mlp.fit(X_train, y_train)

        mlp_predTrain = mlp.predict(X_train)
        mlp_errorTrain = y_train - mlp_predTrain
        printError("MLP","Train", y_train, mlp_predTrain)

        mlp_predVal = mlp.predict(X_val)
        mlp_errorVal = y_val - mlp_predVal
        printError("MLP","Validation", y_val, mlp_predVal)

        mlp_predTest = mlp.predict(X_test)
        mlp_errorTest = y_test- mlp_predTest
        printError("MLP","Test",y_test, mlp_predTest)

        mape_result_array.append(generate_mape_elem(st,"MLP",np.mean(np.abs(mlp_errorTest / y_test)) * 100))


        #LSTM Model

        X_lstm_train = np.expand_dims(X_train, axis = 2)
        X_lstm_val = np.expand_dims(X_val, axis = 2)
        X_lstm_test = np.expand_dims(X_test, axis = 2)
        
        y_lstm_train = np.expand_dims(y_train, axis = 1)
        y_lstm_val = np.expand_dims(y_val, axis = 1)
        y_lstm_test = np.expand_dims(y_test, axis = 1)

        model_lstm = Sequential()
        model_lstm.add(LSTM(36, activation='relu', input_shape=(X_lstm_train.shape[1], X_lstm_train.shape[2])))
        model_lstm.add(Dense(12))
        model_lstm.add(Dense(1))
        model_lstm.compile(optimizer=Adam(0.001), loss='mae')

        lstm_history = model_lstm.fit(X_lstm_train, y_lstm_train, epochs=200,batch_size=15, verbose=2, use_multiprocessing=True)
        
        lstm_predTrain = model_lstm.predict(X_lstm_train) 
        lstm_errorTrain = y_lstm_train - lstm_predTrain
        printError("LSTM","Train", y_lstm_train, lstm_predTrain)

        lstm_predVal = model_lstm.predict(X_lstm_val) 
        lstm_errorVal = y_lstm_val - lstm_predVal
        printError("LSTM","Validation", y_lstm_val, lstm_predVal)

        lstm_predTest = model_lstm.predict(X_lstm_test)
        lstm_errorTest = y_lstm_test - lstm_predTest
        printError("LSTM","Test", y_lstm_test, lstm_predTest)

        mape_result_array.append(generate_mape_elem(st,"LSTM",np.mean(np.abs(lstm_errorTest / y_lstm_test)) * 100))
        lstm_results.append(generate_lstm_result(st,hour_interval,y_lstm_test,lstm_predTest))


        save_error_to_excel(st_result_header,first_run)
        first_run=False
    

    save_column_index+=10
    save_row_index=0


In [ ]:
def plotMAPEFromResults(st):

    decision_tree_results=[]
    mlp_results=[]
    linear_regression_results=[]
    lstm_results=[]
    
    for result_dict in mape_result_array:
        if result_dict["st"] == st:
            if result_dict["model"] == "Decision Tree":
                decision_tree_results.append(result_dict["mape"])
            if result_dict["model"] == "MLP":
                mlp_results.append(result_dict["mape"]) 
            if result_dict["model"] == "Linear Regression":
                linear_regression_results.append(result_dict["mape"])
            if result_dict["model"] == "LSTM":
                lstm_results.append(result_dict["mape"])
    
    plt.figure(figsize=(5,3))
    
    plt.plot(["1","2","6","12","18","24"],linear_regression_results,c='r', label="Lineer Regression")
    plt.plot(["1","2","6","12","18","24"],mlp_results,c='g', label="MLP")
    plt.plot(["1","2","6","12","18","24"],decision_tree_results,c='b', label="Decision Tree")
    plt.plot(["1","2","6","12","18","24"],lstm_results,c='y', label="LSTM")
    
    plt.title(st[:st.find('2')] + "  " + st[st.find('2'):st.rfind('_')] + " MAPEs")
    plt.legend(fontsize="small")
    plt.savefig("graphs/"+st[:st.find('2')] + "  " + st[st.find('2'):st.rfind('_')] + " MAPEs",dpi=500,bbox_inches='tight')   
    plt.show()


#plots hard-coded station lstm results,needs to be generalized
def plot_lstm_preds(st):

    lstm1_real,lstm6_real,lstm24_real,lstm1_pred,lstm6_pred,lstm24_pred=[],[],[],[],[],[]

    
    hour_list=stations_data[st]["Hour"][-120:].astype(int)
    mm_dd_list=stations_data[st]["Date"][-120:].astype(str)
    mm_dd_hh_list= [s1+","+str(s2) for s1, s2 in zip(mm_dd_list, hour_list)]

    step_size = len(mm_dd_hh_list) // 5  # We want to show only ten elements, so divide the total number of elements by 10
    x_ticks_positions = list(range(0, len(mm_dd_hh_list), step_size))
    x_ticks_labels = [day[:-2] for day in mm_dd_hh_list[::step_size]]
    
    
    font = {'family': 'serif',
    'color':  'darkred',
    'weight': 'normal',
    'size': 20
    }

    plt.style.use('default')

    for result_dict in lstm_results:
        if (result_dict["st"] == st) & (result_dict["hour"] == 1):
            lstm1_real=result_dict["real"]
            lstm1_pred=result_dict["pred"]
        if (result_dict["st"] == st) & (result_dict["hour"] == 6):
            lstm6_real=result_dict["real"]
            lstm6_pred=result_dict["pred"]
        if (result_dict["st"] == st) & (result_dict["hour"] == 24):
            lstm24_real=result_dict["real"]
            lstm24_pred=result_dict["pred"]
        
    fig, (ax1,ax2,ax3) = plt.subplots(3,1, sharex=False, sharey=False, figsize=(20,21))

    #-120 means last five days
    ax1.plot(mm_dd_hh_list,lstm1_pred[-120:] , '-s', markersize=4, c="b", linewidth=2.2, label="Predicted")
    ax1.plot(mm_dd_hh_list,lstm1_real[-120:], '-o', markersize=4, c="r", linewidth=2.2,  label="Measured")
    
    ax2.plot(mm_dd_hh_list,lstm6_pred[-120:] , '-s', markersize=4, c="b", linewidth=2.2, label="Predicted")
    ax2.plot(mm_dd_hh_list,lstm6_real[-120:] , '-o', markersize=4, c="r", linewidth=2.2,  label="Measured")

    ax3.plot(mm_dd_hh_list,lstm24_pred[-120:] , '-s', markersize=4, c="b", linewidth=2.2, label="Predicted")
    ax3.plot(mm_dd_hh_list,lstm24_real[-120:],  '-o', markersize=4, c="r", linewidth=2.2,  label="Measured")

    title = st[:st.find('2')] + "  " + st[st.find('2'):st.rfind('_')]
    ax1.set_title(title + ' 1-h ahead foF2 prediction', fontdict=font)
    ax2.set_title(title + ' 12-h ahead foF2 prediction', fontdict=font)
    ax3.set_title(title +' 24-h ahead foF2 prediction', fontdict=font)

    ax1.set_ylabel('foF2 (MHz)', fontsize = 24)
    ax2.set_ylabel('foF2 (MHz)', fontsize = 24)
    ax3.set_ylabel('foF2 (MHz)', fontsize = 24)

    ax3.set_xlabel('Date (mm-dd,hh)', fontsize = 24)

    ax1.grid(True, linestyle='-.')
    ax2.grid(True, linestyle='-.')
    ax3.grid(True, linestyle='--')

    # Set the tick positions and labels for the x-axis
    ax1.set_xticks(x_ticks_positions, x_ticks_labels)
    ax2.set_xticks(x_ticks_positions, x_ticks_labels)
    ax3.set_xticks(x_ticks_positions, x_ticks_labels)

    ax1.tick_params(labelcolor='0', labelsize='14', width=1)
    ax2.tick_params(labelcolor='0', labelsize='14', width=1)
    ax3.tick_params(labelcolor='0', labelsize='14', width=1)

    ax1.legend(ncol=2, loc='upper left', fontsize=18,shadow=True)
    ax2.legend(ncol=2, loc='upper left', fontsize=18,shadow=True)
    ax3.legend(ncol=2, loc='upper left', fontsize=18,shadow=True)

    plt.savefig("graphs/"+title+" Predictions.jpg", format="jpg", dpi=500)  
 


In [ ]:

for st in stations:
    plotMAPEFromResults(st)
    plot_lstm_preds(st) 
